In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10562,2024-10-04,República Tcheca 1. Liga,14:30,Opava 2,Novy Jicin,2.87,1.34,165.5,1.82,1.84,6.5,1.95,1.72,neJeB7qS,0.348432,0.746269,0.549451,0.543478,0.094701,350.334,219.298023,0.625968,0.0,0.000000,NaN,192.78,4.692,2.636744,0.561966,-45.0,132.408,36.622278,0.276587,1.2,1.643168,1.369306,151.11,1.682,0.575039,0.341878,9.0,63,69,3.06,2.19,268.946,680.318,0.513954,0.007728,0.088629,-2.20,-0.440,-4.250000,0.000000,0.0,0.000000,19.00,3.800,0.089474,0.000000,0.0,0.000000
10563,2024-10-04,República Tcheca 1. Liga,14:00,Plzen,Polabi,1.73,1.94,149.5,1.85,1.81,-2.5,1.97,1.71,h0r6bZOr,0.578035,0.515464,0.540541,0.552486,0.093499,177.604,140.309485,0.790013,0.6,1.341641,2.236068,91.30,2.478,1.753160,0.707490,-48.0,126.916,28.433883,0.224037,2.4,1.341641,0.559017,124.63,1.186,0.094499,0.079678,130.0,55,103,1.66,1.21,96.556,493.402,0.080922,0.015456,0.099917,-1.17,-0.234,-3.119658,0.000000,0.0,0.000000,-2.61,-0.522,-1.800766,0.000000,0.0,0.000000
10564,2024-10-04,República Tcheca 1. Liga,14:00,Pardubice 2,Snakes Ostrava,1.46,2.47,153.5,1.83,1.83,-5.5,1.88,1.79,4WHvGpmm,0.684932,0.404858,0.546448,0.546448,0.089790,288.446,208.010865,0.721143,1.8,1.643168,0.912871,322.92,4.106,2.765950,0.673636,-34.0,338.078,137.444892,0.406548,1.2,1.643168,1.369306,164.64,4.316,2.080776,0.482108,-8.0,54,98,5.98,1.68,297.848,360.446,0.363449,0.000000,0.034681,7.53,1.506,0.305445,0.000000,0.0,0.000000,-3.27,-0.654,-2.247706,0.000000,0.0,0.000000
10565,2024-10-04,República Tcheca 1. Liga,14:00,Chomutov,Lynx Liberec,2.52,1.44,142.5,1.83,1.83,4.5,1.96,1.71,lSyFdDge,0.396825,0.694444,0.546448,0.546448,0.091270,119.454,24.018669,0.201070,1.2,1.643168,1.369306,117.00,1.504,0.355007,0.236042,-14.0,220.266,145.138247,0.658923,1.2,1.643168,1.369306,135.29,2.924,1.898613,0.649320,-3.0,75,83,1.56,1.63,122.476,238.446,0.385695,0.000000,0.096336,1.37,0.274,5.547445,0.617954,0.7,0.082046,-5.00,-1.000,-0.440000,0.000000,0.0,0.000000
10566,2024-10-04,Portugal Lpb,16:00,FC Porto,CD Póvoa,1.04,7.50,158.5,1.84,1.82,-17.5,1.97,1.71,UF0aRvVb,0.961538,0.133333,0.543478,0.549451,0.094872,156.598,111.826820,0.714101,1.2,1.643168,1.369306,346.56,2.140,1.507266,0.704330,-1.0,355.426,325.306676,0.915259,0.6,1.341641,2.236068,284.28,4.710,4.224642,0.896952,-43.0,76,69,4.56,4.12,106.334,452.938,1.069768,0.007728,0.099917,-1.55,-0.310,-0.129032,0.874486,0.8,-0.074486,5.49,1.098,5.919854,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10630,2024-10-04,Finlândia Korisliiga,12:30,Helsinki Seagulls,Salon Vilpas,1.23,3.55,164.5,1.83,1.83,-8.5,1.95,1.72,CGT94zSO,0.813008,0.281690,0.546448,0.546448,0.094698,130.436,25.200446,0.193202,1.8,1.643168,0.912871,102.00,1.744,0.384682,0.220574,5.0,187.402,68.724149,0.366720,1.2,1.643168,1.369306,247.42,2.168,0.580491,0.267754,2.0,75,89,1.36,2.78,113.216,201.920,0.686397,0.000000,0.088629,-0.60,-0.120,-1.916667,0.770895,0.7,-0.070895,-2.71,-0.542,-4.704797,0.420307,0.4,-0.020307
10631,2024-10-04,Finlândia Korisliiga,12:30,Bisons Loimaa,UU-Korihait,1.67,2.02,166.5,1.84,1.82,-3.5,2.01,1.68,8OWh8ISa,0.598802,0.495050,0.543478

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10608,16:00,Europa Euroliga,Virtus Bologna,Anadolu Efes,1.87,Back Home
10609,16:00,Europa Euroliga,Paris,Crvena zvezda,2.05,Back Home
10631,12:30,Finlândia Korisliiga,Bisons Loimaa,UU-Korihait,1.67,Back Home
